# Власна реалізація метода QDA
## Завантажимо дані

In [176]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris()
data = pd.DataFrame(iris.data, columns=iris.feature_names)

data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


## Розділимо дані на тренувальні та тестові

In [177]:
from sklearn.model_selection import train_test_split

# 1) Розбиття даних один раз
target = "sepal length (cm)"
X_full = data.drop(columns=[target])
y_full = data[target]
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.3, random_state=4)

df_train = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
df_test = pd.concat([X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)

## Визначимо функцію апріорної вірогідності

In [178]:
def prior_probability(y: pd.Series, class_value) -> float:
    y = pd.Series(y)
    return (y == class_value).mean()

## Визначимо функцію вирівнювання векторів

In [179]:
def _ensure_aligned(X: pd.DataFrame, y: pd.Series) -> tuple[pd.DataFrame, pd.Series]:
    if len(X) != len(y):
        raise ValueError(f"X і y мають різну довжину: len(X)={len(X)}, len(y)={len(y)}")
    if not X.index.equals(y.index):
        X = X.reset_index(drop=True)
        y = y.reset_index(drop=True)
    return X, y

## Визначаємо функцію середнього класу

In [180]:
def class_mean(X: pd.DataFrame, y: pd.Series, class_value) -> np.ndarray:
    X, y = _ensure_aligned(X, y)
    mask = (y == class_value).to_numpy()
    if mask.sum() == 0:
        raise ValueError(f"У класі '{class_value}' немає спостережень.")
    return X.to_numpy()[mask].mean(axis=0)

## Визначаємо функцію коваріації за класом

In [181]:
def class_cov(X: pd.DataFrame, y: pd.Series, class_value, reg: float = 1e-6) -> np.ndarray:
    X, y = _ensure_aligned(X, y)
    mask = (y == class_value).to_numpy()
    if mask.sum() < 2:
        raise ValueError(f"Недостатньо спостережень для оцінки коваріації класу '{class_value}'.")
    Xc = X.to_numpy()[mask]
    sigma = np.cov(Xc, rowvar=False, bias=False)
    return sigma + reg * np.eye(sigma.shape[0])

## Визначаємо функцію лог-дискримінантних оцінок QDA

In [182]:
def qda_discriminant(x: np.ndarray, X: pd.DataFrame, y: pd.Series, class_value) -> float:
    X, y = _ensure_aligned(X, y)
    mu = class_mean(X, y, class_value)
    sigma = class_cov(X, y, class_value)
    sign, logdet = np.linalg.slogdet(sigma)
    if sign <= 0:
        eps = 1e-4
        sigma = sigma + eps * np.eye(sigma.shape[0])
        sign, logdet = np.linalg.slogdet(sigma)
    invS = np.linalg.inv(sigma)
    pi = prior_probability(y, class_value)
    dx = x - mu
    return -0.5 * logdet - 0.5 * float(dx.T @ invS @ dx) + np.log(pi)

## Визначаємо функцію розпаковки даних за певним класом

In [183]:
def get_feature(df: pd.DataFrame, class_name: str, index: int) -> tuple:
    # ВАЖЛИВО: y беремо з того ж df
    X = df.drop(columns=[class_name])
    y = df[class_name]
    x_n = X.iloc[index].to_numpy()
    # Для QDA class_value має бути значенням класу, а не назвою стовпця.
    # У вашому прикладі target — це безперервна змінна (довжина чашолистка),
    # QDA має сенс для дискретних класів. Для демонстрації візьмемо бінінг:
    class_value = (y <= y.median()).iloc[0]  # лише приклад, зазвичай потрібна категоріальна
    return x_n, X, (y <= y.median()), class_value  # порядок під qda_discriminant(x, X, y, ...)

## Функція розпаковки усіх класів за рядком

In [184]:
def all_features(df: pd.DataFrame, index: int)->dict:
    columns = df.columns
    return {column: get_feature(df, column, index) for column in columns}

In [185]:
features = all_features(df=df_train, index=0)
features[list(features.keys())[0]]

(array([1.1, 0.1, 4.3]),
      petal length (cm)  petal width (cm)  sepal length (cm)
 0                  1.1               0.1                4.3
 1                  1.5               0.4                5.7
 2                  4.2               1.5                5.9
 3                  4.6               1.4                6.1
 4                  5.5               1.8                6.5
 ..                 ...               ...                ...
 100                4.7               1.4                7.0
 101                4.4               1.3                6.3
 102                5.8               2.2                6.5
 103                5.8               1.6                7.2
 104                6.7               2.0                7.7
 
 [105 rows x 3 columns],
 0       True
 1      False
 2       True
 3       True
 4       True
        ...  
 100    False
 101     True
 102     True
 103     True
 104     True
 Name: sepal width (cm), Length: 105, dtype: bool,
 np.True_)

In [186]:
# Обчислення дискримінанту для одного спостереження з train
qda_discriminant(*get_feature(df_train, target, 0))

np.float64(0.14548019233202947)

## Випробуємо дискримінантну функцію на одному рядку

In [187]:
for feature in features:
    disc = qda_discriminant(*features[feature])
    print(feature, ":", disc)

sepal width (cm) : -2.47593746930016
petal length (cm) : -0.07607713925198711
petal width (cm) : -0.7888480247104656
sepal length (cm) : 0.14548019233202947


## Тренуємо оцінки дискримінантною функцією та застосовуємо отримані оцінки до тестових даних

In [188]:
def binarize_target_by_median(y: pd.Series) -> pd.Series:
    med = y.median()
    return (y <= med).astype(int)


def qda_fit_using_train(df_train: pd.DataFrame, target_col: str):
    X_train = df_train.drop(columns=[target_col])
    y_train = binarize_target_by_median(df_train[target_col])
    classes = sorted(y_train.unique())
    return X_train, y_train, classes


def qda_apply_with_fn(df_apply: pd.DataFrame, target_col: str, X_train: pd.DataFrame, y_train: pd.Series,
                      classes) -> pd.DataFrame:
    X_apply = df_apply.drop(columns=[target_col])
    res = np.zeros((len(X_apply), len(classes)))
    for i in range(len(X_apply)):
        x_i = X_apply.iloc[i].to_numpy()
        for j, c in enumerate(classes):
            res[i, j] = qda_discriminant(x_i, X_train, y_train, c)
    return pd.DataFrame(res, columns=[f"class={c}" for c in classes], index=df_apply.index)



In [189]:
for target in df_train.columns:
    # "Навчання" на train
    Xtr, ytr, classes = qda_fit_using_train(df_train, target)

    # Дискримінанти на train
    G_train = qda_apply_with_fn(df_train, target, Xtr, ytr, classes)
    G_train.head()

    # Дискримінанти на test (параметри беруться з train)
    G_test = qda_apply_with_fn(df_test, target, Xtr, ytr, classes)
    print(target, ":", G_test.mean(axis=0))

sepal width (cm) : class=0   -0.949562
class=1   -1.643167
dtype: float64
petal length (cm) : class=0   -7.776761
class=1   -1.435447
dtype: float64
petal width (cm) : class=0   -10.643952
class=1    -1.782057
dtype: float64
sepal length (cm) : class=0   -9.104848
class=1   -1.020227
dtype: float64


## Робимо те сае з QuadraticDiscriminantAnalysis() з sklearn.discriminant_analysis

In [192]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

for target in df_train.columns:
    # підготовка train
    Xtr = df_train.drop(columns=[target])
    ytr = binarize_target_by_median(df_train[target])

    # підготовка test
    Xte = df_test.drop(columns=[target])
    yte = binarize_target_by_median(df_test[target])
    # навчання
    qda = QuadraticDiscriminantAnalysis(store_covariance=True)
    qda.fit(Xtr, ytr)

    # лог-дискримінанти (аналог ваших g_c(x))
    G_train = qda.decision_function(Xtr)  # shape (n_train, n_classes)
    G_test  = qda.decision_function(Xte)

    # ймовірності p(c|x)
    P_train = qda.predict_proba(Xtr)
    P_test  = qda.predict_proba(Xte)

    # передбачені класи
    yhat_train = qda.predict(Xtr)
    yhat_test  = qda.predict(Xte)

    print(P_test, ":", P_test.mean(axis=0))

[[2.42659129e-01 7.57340871e-01]
 [9.98589104e-01 1.41089645e-03]
 [1.00697433e-01 8.99302567e-01]
 [8.65101524e-02 9.13489848e-01]
 [9.68694445e-03 9.90313056e-01]
 [1.27880126e-01 8.72119874e-01]
 [1.25346550e-02 9.87465345e-01]
 [9.99937483e-01 6.25171740e-05]
 [9.94531355e-01 5.46864548e-03]
 [1.74278291e-02 9.82572171e-01]
 [9.51843147e-01 4.81568535e-02]
 [8.49199723e-01 1.50800277e-01]
 [9.36581375e-01 6.34186246e-02]
 [3.29565259e-02 9.67043474e-01]
 [1.32466930e-01 8.67533070e-01]
 [9.53967618e-01 4.60323817e-02]
 [8.08412198e-03 9.91915878e-01]
 [9.55812077e-01 4.41879230e-02]
 [3.91331678e-01 6.08668322e-01]
 [3.32876839e-01 6.67123161e-01]
 [9.95865974e-01 4.13402634e-03]
 [3.37726922e-01 6.62273078e-01]
 [2.43444079e-01 7.56555921e-01]
 [9.73300235e-01 2.66997653e-02]
 [7.46631981e-01 2.53368019e-01]
 [9.89073413e-01 1.09265870e-02]
 [8.91534128e-01 1.08465872e-01]
 [9.99250276e-01 7.49724056e-04]
 [9.90915099e-01 9.08490136e-03]
 [6.82052138e-02 9.31794786e-01]
 [1.000497

## Порівнюємо отримані результати та виводимо статистику

In [196]:
def our_scores(X_apply: pd.DataFrame) -> np.ndarray:
    G = np.zeros((len(X_apply), len(classes)))
    for i in range(len(X_apply)):
        x_i = X_apply.iloc[i].to_numpy()
        for j, c in enumerate(classes):
            G[i, j] = qda_discriminant(x_i, Xtr, ytr, c)
    return G

def row_center(G: np.ndarray) -> np.ndarray:
    if G is None:
        raise ValueError("row_center: отримано None замість масиву.")
    G = np.asarray(G)
    if G.ndim == 0:
        raise ValueError("row_center: очікується 1D або 2D масив, отримано скаляр.")
    # Нормалізуємо до 2D: (n, m). 1D вектор стане (n, 1)
    if G.ndim == 1:
        G = G.reshape(-1, 1)
    if G.size == 0:
        return G.copy()
    # Віднімаємо максимум у кожному рядку
    return G - G.max(axis=1, keepdims=True)

def to_class_labels(scores_or_labels):
    arr = np.asarray(scores_or_labels)
    if arr.ndim == 1:
        # або вже класи, або 1D ймовірності/скори
        # якщо значення лише 0/1/цілі — вважати класами
        if np.issubdtype(arr.dtype, np.integer):
            return arr
        # якщо флоати: бінарна класифікація
        # decision_function зазвичай має значення навколо 0
        # якщо діапазон [-inf, inf], застосуємо поріг 0
        # якщо в [0,1], застосуємо поріг 0.5
        vmin, vmax = np.nanmin(arr), np.nanmax(arr)
        thr = 0.0 if (vmin < 0 and vmax > 1) else 0.5
        return (arr >= thr).astype(int)
    elif arr.ndim == 2:
        return arr.argmax(axis=1)
    else:
        raise ValueError("Непідтримувана розмірність виходу: ndim={}".format(arr.ndim))


for target in df_train.columns:
    # 1) Підготовка однакових X,y (класи і пріори з train)
    Xtr = df_train.drop(columns=[target])
    ytr = binarize_target_by_median(df_train[target])

    Xte = df_test.drop(columns=[target])

    # 2) Наші оцінки через qda_discriminant (з параметрами з train)
    classes = sorted(ytr.unique())

    G_train_ours = our_scores(Xtr)
    G_test_ours  = our_scores(Xte)

    # 3) sklearn QDA з тими самими пріорами та без додаткової усадки
    priors = np.array([(ytr == c).mean() for c in classes])

    qda = QuadraticDiscriminantAnalysis(priors=priors, reg_param=0.0, store_covariance=True)
    qda.fit(Xtr, ytr)

    G_train_skl = qda.decision_function(Xtr)  # (n_train, n_classes)
    G_test_skl  = qda.decision_function(Xte)

    Gt_ours_c  = row_center(G_train_ours)
    Gt_skl_c   = row_center(G_train_skl)
    Ge_ours_c  = row_center(G_test_ours)
    Ge_skl_c   = row_center(G_test_skl)

    # 4) Метрики збіжності
    train_abs_diff = np.abs(Gt_ours_c - Gt_skl_c).mean()
    test_abs_diff  = np.abs(Ge_ours_c - Ge_skl_c).mean()

    print("Сер. |різниця| після центрування (train):", train_abs_diff)
    print("Сер. |різниця| після центрування (test):",  test_abs_diff)

    # 6) Опціонально: перевіримо, чи дають однакові класи
    pred_ours_train = to_class_labels(G_train_ours)
    pred_skl_train  = to_class_labels(G_train_skl)
    pred_ours_test  = to_class_labels(G_test_ours)
    pred_skl_test   = to_class_labels(G_test_skl)

    print("Збіг прогнозів (train):", (pred_ours_train == pred_skl_train).mean())
    print("Збіг прогнозів (test):",  (pred_ours_test == pred_skl_test).mean())

Сер. |різниця| після центрування (train): 1.2889492200898975
Сер. |різниця| після центрування (test): 1.5516583070511212
Збіг прогнозів (train): 1.0
Збіг прогнозів (test): 1.0
Сер. |різниця| після центрування (train): 4.355225208626357
Сер. |різниця| після центрування (test): 5.32825079627171
Збіг прогнозів (train): 1.0
Збіг прогнозів (test): 1.0
Сер. |різниця| після центрування (train): 4.621725684646303
Сер. |різниця| після центрування (test): 6.304866966239645
Збіг прогнозів (train): 1.0
Збіг прогнозів (test): 1.0
Сер. |різниця| після центрування (train): 4.224775343174672
Сер. |різниця| після центрування (test): 5.732205171882209
Збіг прогнозів (train): 1.0
Збіг прогнозів (test): 1.0


# Висновок

З проведених експериментів видно, що створена нами дискримінантна функція дала інакші оцінки, аніж модель, імпортована з **sklearn**. Це, вірогідно, повʼязано з такими причинами:

- Попередня обробка цілі: ви робите бінінг за медіаною; sklearn очікує реальні класи. Інший розподіл класів → інші пріори π_c.
- Пріори класів: у sklearn class_prior обчислюється з часток у y (або заданий), у нашому коді це також так, але навіть мінімальні відмінності в бінінгу/фільтрах змінять log π_c.
- Регуляризація/стабілізація: ми додаємо reg до діагоналі (та ще eps при не-позитивній визначеності). У sklearn є власні чисельні стабілізації й параметри reg_param, store_covariance, tol тощо. Різні sigma → інші log|Σ_c| та Σ_c^{-1}.
- Чисельна лінійна алгебра: відмінності в обчисленні logdet/inv (slogdet+inv vs внутрішні розклади), порядку операцій (центрування, масштабування) → невеликі зсуви.

Та попри те, як бачимо, збіг у прогнозуванні на тестових даних у нашому коді та у функції з sklearn все ж збігаються.